In [3]:
@info("Instantiate Qaintellect environment...")
include("../src/Qaintellect.jl")
using .Qaintellect

using Flux, LinearAlgebra
using Flux.Optimise: update!
using IterTools: ncycle

┌ Info: Instantiate Qaintellect environment...
└ @ Main In[3]:1


In [4]:
function output(ψ, Δ)
    @assert length(ψ) == length(Δ)
    dot(ψ, Δ)
end

output (generic function with 1 method)

In [5]:
# construct parametrized circuit
N = 4
rz = RzGate(1.5π)
ps = PhaseShiftGate(0.3)
ry = RyGate(√2)
n = randn(Float64, 3)
n = n/norm(n)
rg = RotationGate(0.2π, n)

cgc = CircuitGateChain{N}([
    single_qubit_circuit_gate(3, HadamardGate(), N),
    controlled_circuit_gate((1, 4), 2, rz, N),
    two_qubit_circuit_gate(2, 3, SwapGate(), N),
    single_qubit_circuit_gate(3, ps, N),
    single_qubit_circuit_gate(3, rg, N),
    single_qubit_circuit_gate(1, ry, N),
])
meas = MeasurementOps{N}([Matrix{Float64}(I, 2^N, 2^N), Hermitian(randn(ComplexF64, 2^N, 2^N))])

c = Circuit(cgc, meas)


    1 ————————•————————————————[Ry]—
              |                     
    2 ———————[Rz]———x———————————————
              |     |               
    3 —[H ]—————————x————[Pϕ]——[Rθ]—
              |                     
    4 ————————•—————————————————————


In [6]:
# input quantum state
ψ = randn(ComplexF64, 2^N)
ψ = ψ/norm(ψ)

# fictitious gradients of cost function with respect to circuit output
Δ = [0.3, -1.2]
e = 0.65

0.65

In [7]:
# set up model
model(ψ) = output(c(ψ), Δ)

# create loss function
loss(x,y) = Flux.mse(model(x), y)

# gather parameters from Circuit
paras = Flux.params(c)

# freeze parameter ry.θ, equivalent to delete!(paras, Qaintessent.get_trainable(ry))
delete!(paras, ry.θ)

# set up data for training
data = ncycle([(ψ, e)], 150)

# define optimizer
opt = Descent(1)

# define evaluation function
evalcb() = @show(loss(ψ, e))

evalcb (generic function with 1 method)

In [8]:
println("Initial model evaluation: " * string(model(ψ)) * ", Target: " * string(e))

Flux.train!(loss, paras, data, opt, cb=Flux.throttle(evalcb, 0.01))

println("Final model evaluation: " * string(model(ψ)) * ", Target: " * string(e))

Initial model evaluation: 0.4166072481798071, Target: 0.65
loss(ψ, e) = 2.133335227886131
loss(ψ, e) = 5.972201643196805
loss(ψ, e) = 6.527247392966332e-7
loss(ψ, e) = 1.2259673922757054e-11
loss(ψ, e) = 6.098230404178701e-16
loss(ψ, e) = 2.2006812507635354e-19
loss(ψ, e) = 7.941551262775014e-23
loss(ψ, e) = 2.8627959465696824e-26
loss(ψ, e) = 3.993608332681372e-30
loss(ψ, e) = 1.232595164407831e-32
loss(ψ, e) = 1.232595164407831e-32
loss(ψ, e) = 1.232595164407831e-32
loss(ψ, e) = 1.232595164407831e-32
loss(ψ, e) = 1.232595164407831e-32
loss(ψ, e) = 1.232595164407831e-32
loss(ψ, e) = 1.232595164407831e-32
loss(ψ, e) = 1.232595164407831e-32
Final model evaluation: 0.6500000000000001, Target: 0.65
